In [ ]:
!pip install healpy
!pip install camb
!pip install emcee
!pip install corner

In [ ]:
import healpy as hp
import camb
import numpy as np
import emcee
import matplotlib.pyplot as plt
import corner

In [ ]:
input_map = "https://irsa.ipac.caltech.edu/data/Planck/release_2/all-sky-maps/maps/component-maps/cmb/COM_CMB_IQU-commander_1024_R2.02_full.fits"
cmb_map = hp.read_map(input_map)

In [ ]:
lmax = 2500  # Maximum multipole to consider
cl = hp.anafast(cmb_map, lmax=lmax)

In [ ]:
def get_theoretical_spectrum(params, lmax):
    H0, ombh2, omch2, As, ns, tau = params
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0, ombh2=ombh2, omch2=omch2, mnu=0.06, omk=0, tau=tau)
    pars.InitPower.set_params(As=As, ns=ns, r=0)
    pars.set_for_lmax(lmax, lens_potential_accuracy=0)

    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars, CMB_unit="muK")
    return powers["total"][0:(lmax+1), 0]  # Start from l=2

In [ ]:
def ln_likelihood(params, cl_data, lmax):
    cl_theory = get_theoretical_spectrum(params, lmax)

    # Check for NaN or infinite values in the power spectrum
    if np.any(np.isnan(cl_theory)) or np.any(np.isinf(cl_theory)):
        return -np.inf

    epsilon = 1e-12
    diff = cl_data - cl_theory
    chi2 = np.sum(diff**2 / (cl_theory + epsilon))
    return -0.5 * chi2

def ln_prior(params):
    H0, ombh2, omch2, As, ns, tau = params

    # Check if any of the parameters are NaN or infinite
    if np.any(np.isnan(params)) or np.any(np.isinf(params)):
        return -np.inf

    # Update the bounds for the parameters if necessary
    if 40 < H0 < 100 and 0.005 < ombh2 < 0.1 and 0.01 < omch2 < 0.9 and 1e-10 < As < 1e-8 and 0.8 < ns < 1.2 and 0.01 < tau < 0.2:
        return 0.0
    return -np.inf

def ln_posterior(params, cl_data, lmax):
    lp = ln_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + ln_likelihood(params, cl_data, lmax)

In [ ]:
nwalkers = 100000
ndim = 6
nsteps = 100

initial_params = np.array([70, 0.022, 0.12, 2.1e-9, 0.96, 0.06])
noise_scale = 5  # Set the noise scale factor
#initial_params_scale = np.array([5, 0.1, 0.8, 1e-8, 0.3, 0.15])
#initial_params_scale = noise_scale * initial_params
pos = initial_params + noise_scale * np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, args=(cl, lmax));
sampler.run_mcmc(pos, nsteps, progress=True);
print('\nDone')

In [ ]:
# Discard burn-in samples and flatten the chain
burn_in = 90
samples = sampler.get_chain(discard=burn_in, flat=True)

# Extract H_0 samples
H0_samples = samples[:, 0]

# Calculate the mean and 1-sigma confidence intervals
H0_mean = np.mean(H0_samples)
H0_std = np.std(H0_samples)
H0_lower = H0_mean - H0_std
H0_upper = H0_mean + H0_std

print(f"H_0 = {H0_mean:.2f} + {H0_upper - H0_mean:.2f} - {H0_mean - H0_lower:.2f} km/s/Mpc")

# Plot the posterior distributions for all parameters
labels = ["H_0", "Ω_b h^2", "Ω_c h^2", "A_s", "n_s", "τ"]
corner.corner(samples, labels=labels, show_titles=True)
plt.show()